# SPCS Networking Connectivity Test: AWS Kinesis

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Please provide your AWS Region
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.


In [ ]:
# AWS Kinesis Connectivity Test Configuration
# Update the AWS_REGION with your actual AWS region

# User Configuration
AWS_REGION = "us-west-2"  # Replace with your AWS region (e.g., us-east-1, eu-west-1, etc.)

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"


OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

# Determine if FIPS endpoints are available (US regions only)
IS_US_REGION = AWS_REGION.startswith('us-')

# Construct AWS service endpoints based on region
KINESIS_ENDPOINT = f"https://kinesis.{AWS_REGION}.amazonaws.com"
KINESIS_API_ENDPOINT = f"https://kinesis.{AWS_REGION}.api.aws"
DYNAMODB_ENDPOINT = f"https://dynamodb.{AWS_REGION}.amazonaws.com"
CLOUDWATCH_ENDPOINT = f"https://monitoring.{AWS_REGION}.amazonaws.com"

print(f"Configuration:")
print(f"AWS Region: {AWS_REGION}")
print(f"FIPS Endpoints: {'Enabled' if IS_US_REGION else 'Disabled (non-US region)'}")
print(f"Kinesis Endpoint: {KINESIS_ENDPOINT}")
print(f"DynamoDB Endpoint: {DYNAMODB_ENDPOINT}")
print(f"CloudWatch Endpoint: {CLOUDWATCH_ENDPOINT}")
print("\nReady to test connectivity...")


## Error examples

This is the kind of error you will get in Openflow if your EAI is not setup correctly

```
16:22:16 GMT
ERROR
a5b9654b-0199-1000-0000-000072809dc0
All Nodes
PutKinesisStream[id=a5b9654b-0199-1000-0000-000072809dc0] Failed to publish due to exception Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long. flowfiles [StandardFlowFileRecord[uuid=eb6fd670-8b6d-464d-8a41-6760af02814d,claim=StandardContentClaim [resourceClaim=StandardResourceClaim[id=1759338026442-113, container=repo1, section=113], offset=663980, length=2565],offset=0,name=eb6fd670-8b6d-464d-8a41-6760af02814d,size=2565]] 
```

## Connectivity Test

Run this test cell to check if you have Network Connectivity to Kinesis.  
If it fails, use the following cells to create and attach the EAI to this Notebook, restart the Notebook session and then test again.

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\nTesting {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("AWS KINESIS CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['kinesis'] = test_connectivity(KINESIS_ENDPOINT, "Kinesis Service")
results['kinesis_api'] = test_connectivity(KINESIS_API_ENDPOINT, "Kinesis API")
results['dynamodb'] = test_connectivity(DYNAMODB_ENDPOINT, "DynamoDB Service")
results['cloudwatch'] = test_connectivity(CLOUDWATCH_ENDPOINT, "CloudWatch Monitoring")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "PASS" if passed else "FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\nAll connectivity tests PASSED!")
    print("Your SPCS environment can reach all required AWS Kinesis endpoints.")
    print("You can proceed with Kinesis integration.")
else:
    print(f"\nSome connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")


If Connectivity testing fails, you can use the Cells below to prepare and implement an EAI suitable for AWS Kinesis access.


In [ ]:
-- Create Network Rule for AWS Kinesis connectivity
-- Run this cell if connectivity tests failed
-- Uses the AWS_REGION variable defined in the configuration cell above

USE ROLE {{IMPLEMENTATION_ROLE}};

CREATE OR REPLACE NETWORK RULE kinesis_access_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = (
    -- Standard endpoints (all regions)
    'kinesis.{{ AWS_REGION }}.amazonaws.com:443',
    'kinesis.{{ AWS_REGION }}.api.aws:443',
    '*.control-kinesis.{{ AWS_REGION }}.amazonaws.com:443',
    '*.control-kinesis.{{ AWS_REGION }}.api.aws:443',
    '*.data-kinesis.{{ AWS_REGION }}.amazonaws.com:443',
    '*.data-kinesis.{{ AWS_REGION }}.api.aws:443',
    'dynamodb.{{ AWS_REGION }}.amazonaws.com:443',
    'monitoring.{{ AWS_REGION }}.amazonaws.com:80',
    'monitoring.{{ AWS_REGION }}.amazonaws.com:443',
    'monitoring.{{ AWS_REGION }}.api.aws:80',
    'monitoring.{{ AWS_REGION }}.api.aws:443'{% if IS_US_REGION %},
    -- FIPS endpoints (US regions only)
    'kinesis-fips.{{ AWS_REGION }}.api.aws:443',
    'kinesis-fips.{{ AWS_REGION }}.amazonaws.com:443',
    'monitoring-fips.{{ AWS_REGION }}.amazonaws.com:80',
    'monitoring-fips.{{ AWS_REGION }}.amazonaws.com:443'{% endif %}
  );

SHOW NETWORK RULES LIKE 'kinesis_%';


In [ ]:
-- Create External Access Integration for AWS Kinesis
-- This uses the network rule created above

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION kinesis_eai
  ALLOWED_NETWORK_RULES = (kinesis_access_rule)
  ENABLED = TRUE
  COMMENT = 'External Access Integration for AWS Kinesis connectivity';

-- Grant usage on the integration to your roles
GRANT USAGE ON INTEGRATION kinesis_eai TO ROLE {{IMPLEMENTATION_ROLE}};
GRANT USAGE ON INTEGRATION kinesis_eai TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'kinesis_eai';


In [ ]:
ALTER NOTEBOOK EAI_KINESIS
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('KINESIS_EAI');


After Creating and Setting the EAI on the Notebook, restart your Notebook session and run the Connectivity test again.
